# Imports

In [1]:
import os

import numpy as np

from scripts.voice_recognition_system import VoiceRecognitionSystem

# Define File Paths

In [2]:
parent_directory = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
data_directory = os.path.join(parent_directory, "data")

In [3]:
database_path = os.path.join(data_directory, "database")
results_path = os.path.join(parent_directory, "results")

incoming_authorized_users_path = os.path.join(
    data_directory, "database", "incoming_users", "authorized_users"
)
incoming_unauthorized_users_path = os.path.join(
    data_directory, "database", "incoming_users", "unauthorized_users"
)

# Initialize System

In [4]:
voice_recognition_system = VoiceRecognitionSystem(
    database_path, acceptance_threshold=0.7, allow_brute_force=False
)

Initializing database: 100%|██████████| 100/100 [00:20<00:00,  4.87it/s]


# Testing on Multiple Users

### Authorized without brute force

In [5]:
df_authorized_users = voice_recognition_system.verify_multiple_users(
    incoming_users_path=incoming_authorized_users_path
)

Veryfing multiple users: 100%|██████████| 50/50 [02:26<00:00,  2.92s/it]


### Unauthorized with brute force

In [6]:
voice_recognition_system.allow_brute_force = True
df_unauthorized_users = voice_recognition_system.verify_multiple_users(
    incoming_users_path=incoming_unauthorized_users_path
)

Veryfing multiple users: 100%|██████████| 50/50 [01:30<00:00,  1.81s/it]


# Calculating metrics

In [7]:
false_acceptance_rate, false_rejection_rate = (
    voice_recognition_system.calculate_far_frr(
        df_authorized_users, df_unauthorized_users
    )
)
true_negative, false_positive, false_negative, true_positive = (
    voice_recognition_system.calculate_ROC_curve(
        df_authorized_users,
        df_unauthorized_users,
        roc_curve_path=os.path.join(results_path, "ROC_curve.png"),
    )
)
accuracy = (true_positive + true_negative) / (
    true_positive + true_negative + false_positive + false_negative
)
access_granted_rate = voice_recognition_system.calculate_access_granted_rate(
    df_authorized_users
)

In [8]:
results = {
    "FAR": np.round(false_acceptance_rate, 3),
    "FRR": np.round(false_rejection_rate, 3),
    "TN": true_negative,
    "FP": false_positive,
    "FN": false_negative,
    "TP": true_positive,
    "ACC": np.round(accuracy, 3),
    "AGR": np.round(access_granted_rate, 3),
}

In [9]:
results

{'FAR': 0.148,
 'FRR': 0.045,
 'TN': 1518,
 'FP': 263,
 'FN': 86,
 'TP': 1832,
 'ACC': 0.906,
 'AGR': 0.955}